# Actividad 3. - Aprendizaje supervisado y no supervisado"
# Alumno: Víctor Eduardo Pérez Aguilar, Matricula: A01796394

# Parte 1.

## Introducción Teórica

Machine learning es una rama de la inteligencia artificial que busca que las computadoras aprendan a partir de datos para tomar decisiones o hacer predicciones, sin que tengamos que programarlas paso a paso para cada tarea. Es como enseñarles a “aprender” de la experiencia, tal como lo hacemos los humanos.

Dentro del aprendizaje automático, existen dos enfoques principales:

- **Aprendizaje supervisado**
- **Aprendizaje no supervisado**

---

## Aprendizaje Supervisado

En el aprendizaje supervisado, le damos al modelo un conjunto de datos donde ya conocemos la respuesta correcta: es decir, tenemos las entradas (las características) y las salidas esperadas (las etiquetas). El objetivo es que el modelo descubra cómo relacionar esas entradas con las salidas para luego poder predecir resultados cuando reciba datos nuevos que no ha visto antes.

Algunos algoritmos comunes en este tipo de aprendizaje son:

- **Decision Tree:** Funcionan como un árbol de preguntas que ayudan a clasificar o predecir un resultado.
- **Random Forest:** Son un conjunto de árboles de decisión que trabajan juntos para mejorar la precisión y evitar errores.
- **GBTClassifier:** Construye varios árboles uno tras otro, donde cada árbol intenta corregir los errores del anterior.
- **Multilayer Perceptron:** Es una red neuronal con varias capas que puede capturar relaciones complejas y no lineales entre las variables.

---

## Aprendizaje No Supervisado

En este caso, el modelo recibe solo los datos de entrada, sin saber las respuestas correctas. La idea es que el modelo encuentre patrones, agrupaciones o estructuras escondidas dentro de esos datos.

Algunos algoritmos populares para este enfoque son:

- **K-means:** Agrupa los datos en “k” grupos basados en qué tan similares son entre sí.
- **Gaussian Mixture:** Supone que los datos vienen de varias distribuciones gaussianas mezcladas, lo que permite una agrupación más flexible.
- **Power Iteration Clustering:** Utiliza técnicas matemáticas avanzadas para agrupar datos según su similitud.

---

## Algoritmos en PySpark

PySpark nos ayuda a procesar grandes volúmenes de datos de forma rápida y distribuida.

Dentro de PySpark está la biblioteca **MLlib**, que incluye implementaciones de muchos de los algoritmos que mencionamos, tanto para aprendizaje supervisado como no supervisado. Esto hace que sea mucho más fácil trabajar con grandes conjuntos de datos y crear modelos eficientes.

Algoritmos que puedes usar en PySpark son:

- Árboles de decisión: DecisionTreeClassifier.
- Bosques aleatorios: RandomForestClassifier.
- Clasificador Boosted por Gradiente: GBTClassifier.
- Perceptrón multicapa: MultilayerPerceptronClassifier.
- K-means
- Gaussian Mixture
- Power Iteration Clustering (PIC)

---


In [1]:
import os
from pyspark.sql import SparkSession

# Inicia una sesión de Spark
spark = SparkSession.builder \
    .appName("Leer CSV Localmente") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Verifica si el archivo existe en la ruta proporcionada
ruta_archivo = "/Users/vpereza/Downloads/DataSetCrimeChicago/Chicago_Crimes_-_2001_to_Present.csv"
if os.path.exists(ruta_archivo):
    print(f"El archivo existe: {ruta_archivo}")
else:
    print(f"El archivo no se encuentra en la ruta: {ruta_archivo}")

# Lee el archivo CSV en un DataFrame de Spark
df = spark.read.option("header", "true").option("inferSchema", "true").csv(ruta_archivo)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/25 10:16:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


El archivo existe: /Users/vpereza/Downloads/DataSetCrimeChicago/Chicago_Crimes_-_2001_to_Present.csv


In [2]:
spark = SparkSession.builder \
    .appName("Chicago Crimes Analysis") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

25/05/25 10:16:28 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Número de registros y columnas
num_registros = df.count()
num_columnas = len(df.columns)

print(f"Número de registros: {num_registros}")
print(f"Número de columnas: {num_columnas}")

# Tipos de datos de cada columna
df.printSchema()

# Ver las estadísticas generales de las columnas numéricas
df.describe().show()

# Identificar valores faltantes (nulos) en cada columna
from pyspark.sql.functions import col, sum

# Crear una lista de expresiones para contar los valores nulos en cada columna
nulos_por_columna = [sum(col(c).isNull().cast("int")).alias(c) for c in df.columns]

# Realizar la agregación para contar los nulos por columna
df.select(nulos_por_columna).show()

Número de registros: 7811711
Número de columnas: 22
root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



25/05/25 10:16:32 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+-------+------------------+------------------+--------------------+--------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+--------------------+-------------------+--------------------+--------------------+
|summary|                ID|       Case Number|                Date|         Block|              IUCR|     Primary Type|    Description|Location Description|              Beat|          District|              Ward|    Community Area|          FBI Code|     X Coordinate|      Y Coordinate|              Year|          Updated On|           Latitude|           Longitude|            Location|
+-------+------------------+------------------+--------------------+--------------+------------------+-----------------+---------------+--------------------+------------------+------------------+------------------+--

[Stage 8:============================================>            (11 + 3) / 14]

+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
| ID|Case Number|Date|Block|IUCR|Primary Type|Description|Location Description|Arrest|Domestic|Beat|District|  Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|Updated On|Latitude|Longitude|Location|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+------------+----+----------+--------+---------+--------+
|  0|          4|   0|    0|   0|           0|          0|               10558|     0|       0|   0|      47|614848|        613476|       0|       87465|       87465|   0|         0|   87465|    87465|   87465|
+---+-----------+----+-----+----+------------+-----------+--------------------+------+--------+----+--------+------+--------------+--------+------------+---

In [7]:
from pyspark.sql.functions import col
from prettytable import PrettyTable

# Crear la tabla de PrettyTable
table = PrettyTable()
table.field_names = ["Columna", "Valores Nulos"]

# Revisar valores nulos y agregarlos a la tabla
for column in df.columns:
    null_count = df.filter(col(column).isNull()).count()
    table.add_row([column, null_count])

# Imprimir la tabla con los valores nulos
print(table)

[Stage 91:========================================>               (10 + 4) / 14]

+----------------------+---------------+
|       Columna        | Valores Nulos |
+----------------------+---------------+
|          ID          |       0       |
|     Case Number      |       4       |
|         Date         |       0       |
|        Block         |       0       |
|         IUCR         |       0       |
|     Primary Type     |       0       |
|     Description      |       0       |
| Location Description |     10558     |
|        Arrest        |       0       |
|       Domestic       |       0       |
|         Beat         |       0       |
|       District       |       47      |
|         Ward         |     614848    |
|    Community Area    |     613476    |
|       FBI Code       |       0       |
|     X Coordinate     |     87465     |
|     Y Coordinate     |     87465     |
|         Year         |       0       |
|      Updated On      |       0       |
|       Latitude       |     87465     |
|      Longitude       |     87465     |
|       Location

In [6]:
from pyspark import SparkConf
from pyspark.sql import SparkSession

# Crear una configuración de Spark
conf = SparkConf().set("spark.sql.warehouse.dir", "/path/to/your/warehouse")

# Deshabilitar los warnings
conf.set("spark.ui.showConsoleProgress", "false")

# Crear la sesión Spark con la configuración
spark = SparkSession.builder.config(conf=conf).getOrCreate()

25/04/27 15:43:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Caracterización Estadística de las Variables

- **Identificadores** (ID, Case Number): Son campos únicos que permiten distinguir cada registro, pero no aportan valor analítico, por lo que se excluyen del análisis estadístico.
- **Fechas** (Date, Year): Se analizan considerando rango, frecuencias y tendencias por año o mes.
- **Variables categóricas** (Primary Type, Location Description): Se reportan frecuencias absolutas y relativas, moda y el número de categorías distintas.
- **Booleanas** (Arrest, Domestic): Se muestran proporciones de verdadero/falso.
- **Numéricas discretas** (District, Ward, Beat, Community Area): Se calculan media, moda, mínimo, máximo y número de valores únicos.
- **Numéricas continuas** (Latitude, Longitude, X/Y Coordinate): Se reportan media, desviación estándar, mínimo y máximo, reflejando la distribución geoespacial de los datos.

Este análisis permite entender el comportamiento general del conjunto de datos y fundamenta las fases de particionamiento y muestreo de la investigación.

In [16]:
df_pandas_T = df_pandas.T
display(df_pandas_T.style.background_gradient(cmap='YlOrRd'))

,0,1,2,3,4
summary,count,mean,stddev,min,max
ID,7811711,7047235.09462818,3514586.1451125084,634,13096706
Case Number,7811707,314071.94444444444,132968.524780014,.JB299184,ZZZ199957
Date,7811711,None,None,01/01/2001 01:00:00 AM,12/31/2022 12:59:00 PM
Block,7811711,None,None,0000X E 100 PL,XX UNKNOWN
IUCR,7811711,1120.9696163074661,811.5586530819171,0110,9901
Primary Type,7811711,None,None,ARSON,WEAPONS VIOLATION
Description,7811711,None,None,$300 AND UNDER,WIREROOM/SPORTS
Location Description,7801153,None,None,"""CTA """"L"""" PLATFORM""",YMCA
Beat,7811711,1185.8247616943331,703.1754648166113,111,2535


In [8]:
from pyspark.sql.functions import countDistinct

columnas = [
    "IUCR", 
    "Primary Type", 
    "Description", 
    "Location Description", 
    "Beat", 
    "District", 
    "Ward", 
    "Community Area", 
    "FBI Code"
]

for colname in columnas:
    print(f"Únicos en '{colname}':")
    df.select(countDistinct(colname)).show()

Únicos en 'IUCR':


+--------------------+
|count(DISTINCT IUCR)|
+--------------------+
|                 404|
+--------------------+

Únicos en 'Primary Type':


+----------------------------+
|count(DISTINCT Primary Type)|
+----------------------------+
|                          36|
+----------------------------+

Únicos en 'Description':


+---------------------------+
|count(DISTINCT Description)|
+---------------------------+
|                        546|
+---------------------------+

Únicos en 'Location Description':


+------------------------------------+
|count(DISTINCT Location Description)|
+------------------------------------+
|                                 216|
+------------------------------------+

Únicos en 'Beat':


...

+------------------------+
|count(DISTINCT District)|
+------------------------+
|                      24|
+------------------------+

Únicos en 'Ward':


+--------------------+
|count(DISTINCT Ward)|
+--------------------+
|                  50|
+--------------------+

Únicos en 'Community Area':


+------------------------------+
|count(DISTINCT Community Area)|
+------------------------------+
|                            78|
+------------------------------+

Únicos en 'FBI Code':


[Stage 142:=======================================>               (10 + 4) / 14]

+------------------------+
|count(DISTINCT FBI Code)|
+------------------------+
|                      26|
+------------------------+



In [9]:
# Top 10 DESCRIPTIONS
print("Top 10 Description:")
df.groupBy("Description").count().orderBy(col("count").desc()).show(10, truncate=False)

# Top 10 LOCATION DESCRIPTION
print("Top 10 Location Description:")
df.groupBy("Location Description").count().orderBy(col("count").desc()).show(10, truncate=False)

# Top 10 PRIMARY TYPE
print("Top 10 Primary Type:")
df.groupBy("Primary Type").count().orderBy(col("count").desc()).show(10, truncate=False)

Top 10 Description:


+----------------------------+------+
|Description                 |count |
+----------------------------+------+
|SIMPLE                      |916750|
|$500 AND UNDER              |632821|
|DOMESTIC BATTERY SIMPLE     |610676|
|TO VEHICLE                  |434447|
|OVER $500                   |416786|
|TO PROPERTY                 |411525|
|AUTOMOBILE                  |299471|
|FORCIBLE ENTRY              |284651|
|POSS: CANNABIS 30GMS OR LESS|278139|
|FROM BUILDING               |255295|
+----------------------------+------+
only showing top 10 rows

Top 10 Location Description:


+------------------------------+-------+
|Location Description          |count  |
+------------------------------+-------+
|STREET                        |2034594|
|RESIDENCE                     |1309164|
|APARTMENT                     |884477 |
|SIDEWALK                      |730172 |
|OTHER                         |270019 |
|PARKING LOT/GARAGE(NON.RESID.)|202991 |
|ALLEY                         |173490 |
|SMALL RETAIL STORE            |147978 |
|SCHOOL, PUBLIC, BUILDING      |146387 |
|RESIDENCE-GARAGE              |135543 |
+------------------------------+-------+
only showing top 10 rows

Top 10 Primary Type:


[Stage 154:===========================================>           (11 + 3) / 14]

+-------------------+-------+
|Primary Type       |count  |
+-------------------+-------+
|THEFT              |1647915|
|BATTERY            |1427736|
|CRIMINAL DAMAGE    |890391 |
|NARCOTICS          |748161 |
|ASSAULT            |509714 |
|OTHER OFFENSE      |485121 |
|BURGLARY           |425159 |
|MOTOR VEHICLE THEFT|378395 |
|DECEPTIVE PRACTICE |346770 |
|ROBBERY            |293311 |
+-------------------+-------+
only showing top 10 rows



In [12]:
from pyspark.sql.functions import col, round

total = df.count()

for colname in ["Arrest", "Domestic"]:
    print(f"\nFrecuencias de {colname}:")
    # Absolutas y relativas
    df.groupBy(colname).count()\
        .withColumn('porcentaje', round((col('count') / total) * 100,2))\
        .orderBy(colname).show()


Frecuencias de Arrest:


+------+-------+----------+
|Arrest|  count|porcentaje|
+------+-------+----------+
| false|5773604|     73.91|
|  true|2038107|     26.09|
+------+-------+----------+


Frecuencias de Domestic:


[Stage 229:=======================================>               (10 + 4) / 14]

+--------+-------+----------+
|Domestic|  count|porcentaje|
+--------+-------+----------+
|   false|6730624|     86.16|
|    true|1081087|     13.84|
+--------+-------+----------+



| Nombre Variable       | Dominio/Valores típicos             | Estadística comportamental                                       | Comentarios adicionales                                 |
|-----------------------|-------------------------------------|------------------------------------------------------------------|---------------------------------------------------------|
| **ID**                | 7,811,711 valores únicos            | min: 1, max: 7,811,711                                           | Identificador único, no aporta valor analítico          |
| **Case Number**       | 7,811,707 valores únicos            | min: JB299184, max: ZZZ199957                                    | Identificador alfanumérico, uso administrativo          |
| **Date**              | 01/01/2001 a 12/31/2022             | min: 01/01/2001 01:00 AM, max: 12/31/2022 12:59 PM               | Rango temporal del dataset                              |
| **Block**             | Ejemplo: 0000X E 100 PL, XX UNKNOWN | —                                                                | Ubicación aproximada, puede tener valores ambiguos      |
| **IUCR**              | min: 0110, max: 09901, únicos: 404  | media: 120.97, std: 466.18, moda: 0820 (632,836 veces)           | Código policial para tipo de crimen                     |
| **Primary Type**      | Ej: ARSON, WEAPONS VIOLATION, #36   | Top: THEFT, moda: THEFT (1,647,915 veces)                        | Segmenta tipo de crimen; fundamental para análisis      |
| **Description**       | Ej: $300 AND UNDER, únicos: 546      | Top: SIMPLE, moda: SIMPLE (916,750 veces)                        | Detalle específico del crimen                           |
| **Location Desc.**    | Ej: "CTA 'L' PLATFORM", únicos: 216  | Top: STREET, moda: STREET (2,034,594 veces)                      | Contexto espacial/social                                |
| **Arrest**            | True/False                          | True 26.08%, False 73.92%, moda: False (5,773,604 veces)         | Tasa de detenciones; analiza efectividad policial       |
| **Domestic**          | True/False                          | True 13.8%, False 86.2%, moda: False (6,730,624 veces)           | Identifica delitos domésticos                           |
| **Beat**              | min: 111, max: 2535, únicos: 305    | media: 1185.82, std: 1703.17, moda: 421 (60,858 veces)           | Unidad básica geográfica de patrullaje                  |
| **District**          | min: 1, max: 31, únicos: 24         | media: 11.29, std: 6.95, moda: 8 (524,899 veces)                 | Zona administrativa policial                            |
| **Ward**              | min: 1, max: 50, únicos: 50         | media: 22.75, std: 13.85, moda: NULL (614,848 veces)\*           | División político-administrativa (muchos nulos)\*       |
| **Community Area**    | min: 1, max: 77, únicos: 78         | media: 37.48, std: 21.54, moda: NULL (613,476 veces)\*           | Área oficial de Chicago (muchos nulos)\*                |
| **FBI Code**          | min: 01A, max: 26, únicos: 26       | media: 12.08, std: 7.31, moda: 06 (1,647,915 veces)              | Código estándar del FBI                                 |
| **X Coordinate**      | min: 1025119, max: 1205159          | media: 1164603.29, std: 416840.03                                | Coordenada geográfica en sistema de Chicago             |
| **Y Coordinate**      | min: 1622, max: 1951622             | media: 1885786.97, std: 889432.27                                | Coordenada geográfica en sistema de Chicago             |
| **Year**              | 2001–2023                           | media: 2009.99, std: 6.29, moda: (completar)                     | Temporalidad del dato                                   |
| **Updated On**        | 01/01/2007 07:32 AM a 12/31/2022... | —                                                                | Fecha de actualización                                  |
| **Latitude**          | min: 36.62, max: 42.02              | media: 41.84, std: 0.09                                         | Coordenada geográfica                                   |
| **Longitude**         | min: -91.69, max: -87.52            | media: -87.67, std: 0.06                                        | Coordenada geográfica                                   |
| **Location**          | Ej: (36.6194…, -91.6865…)           | —                                                                | Combina latitud y longitud                              |

\* Muchos registros tienen valor nulo en Ward y Community Area.


# Parte 2.
## Selección de los datos:

En esta etapa, el objetivo es trabajar con una muestra de tamaño manejable del dataset original, ya que procesar toda la base de datos sería muy costoso en tiempo y recursos. Para lograrlo, primero se definieron particiones relevantes usando las variables **“Primary Type”** y **“Domestic”**, que agrupan los registros según el tipo de crimen y si estuvo o no relacionado con violencia doméstica.

En vez de analizar todos los datos posibles, se extrajeron únicamente los 5 tipos de delito más frecuentes y para cada uno se separaron las instancias en casos **domésticos** y **no domésticos**. Luego, para mantener un tamaño razonable pero representativo de cada grupo, se tomó una **muestra aleatoria del 10%** de cada partición, usando la misma técnica de muestreo propuesta en actividades anteriores.

Finalmente, se unieron todas estas submuestras para formar una **muestra total** sobre la que se llevarán a cabo los análisis y modelos de *machine learning*, garantizando así que el conjunto de trabajo sea diverso, significativo y eficiente de procesar.

Esta aproximación equilibra la necesidad de **velocidad y manejabilidad** con la de obtener **datos representativos** para todas las situaciones relevantes del fenómeno delictivo.


In [4]:
from pyspark.sql.functions import col

# Obtener los 5 tipos de crimen más frecuentes
top_types = [row['Primary Type'] for row in df.groupBy('Primary Type').count()
             .orderBy('count', ascending=False).limit(5).collect()]

# Para cada tipo mas frecuente crear las particiones por 'Domestic' (True/False)
muestras = []
for t in top_types:
    for domestic in [True, False]:
        subgrupo = df.filter((col("Primary Type") == t) & (col("Domestic") == domestic))
        # Se toma muestra del 10% de cada grupo, usando semilla fija para reproducibilidad
        muestra = subgrupo.sample(fraction=0.1, seed=42)
        muestras.append(muestra)
        print(f"Primary Type: {t} | Domestic: {domestic} | Total: {subgrupo.count()} | Muestra: {muestra.count()}")

# Unir todas las submuestras en un solo dataframe
from functools import reduce
from pyspark.sql import DataFrame
muestra_total = reduce(DataFrame.unionAll, muestras)
print(f"\nTamaño total de la muestra final: {muestra_total.count()}")

Primary Type: THEFT | Domestic: True | Total: 44739 | Muestra: 4391


Primary Type: THEFT | Domestic: False | Total: 1603176 | Muestra: 160537


Primary Type: BATTERY | Domestic: True | Total: 626184 | Muestra: 62739


Primary Type: BATTERY | Domestic: False | Total: 801552 | Muestra: 80409


Primary Type: CRIMINAL DAMAGE | Domestic: True | Total: 76172 | Muestra: 7510


...

Primary Type: NARCOTICS | Domestic: False | Total: 747849 | Muestra: 74833


Primary Type: ASSAULT | Domestic: True | Total: 116731 | Muestra: 11674


Primary Type: ASSAULT | Domestic: False | Total: 392983 | Muestra: 39349


[Stage 74:====================================================> (135 + 5) / 140]


Tamaño total de la muestra final: 522964


In [7]:
muestra_total.limit(5).toPandas()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,10226106,HY413535,09/07/2015 03:00:00 AM,032XX N OCTAVIA AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,False,True,...,36,17,06,1126857,1920644,2015,02/10/2018 03:50:01 PM,41.938579,-87.809186,"(41.938578988, -87.809186242)"
1,10228279,HY415980,09/08/2015 05:00:00 PM,053XX S CORNELL AVE,0810,THEFT,OVER $500,RESIDENCE,False,True,...,5,41,06,1188157,1870260,2015,02/10/2018 03:50:01 PM,41.799074,-87.585509,"(41.799073935, -87.585509129)"
2,11859193,JC470832,10/13/2019 05:00:00 AM,076XX S ADA ST,0810,THEFT,OVER $500,SIDEWALK,False,True,...,17,71,06,1168768,1853958,2019,10/20/2019 04:03:03 PM,41.754780,-87.657083,"(41.754780109, -87.657082931)"
3,10236438,HY424246,09/15/2015 11:20:00 AM,091XX S UNIVERSITY AVE,0810,THEFT,OVER $500,STREET,True,True,...,8,47,06,1185476,1844552,2015,02/10/2018 03:50:01 PM,41.728592,-87.596149,"(41.728592358, -87.59614881)"
4,10238624,HY426363,09/16/2015 10:35:00 PM,049XX S FORRESTVILLE AVE,0820,THEFT,$500 AND UNDER,APARTMENT,False,True,...,4,38,06,1180761,1872355,2015,02/10/2018 03:50:01 PM,41.804996,-87.612567,"(41.804996159, -87.612567255)"


## Parte 3: Selección de columnas clave

Se seleccionaron las columnas más importantes para hacer un **análisis general** de los delitos. La idea en esta etapa no es entrar en temas muy específicos, es solo tener una vista amplia del comportamiento delictivo.
Las columnas seleccionadas fueron:

- **Primary Type**: Indica el tipo de crimen. Es la base para clasificar los delitos y entender qué tipos son más frecuentes.
- **Domestic**: Muestra si el delito estuvo relacionado con violencia doméstica. Esto permite separar casos con una dinámica social distinta.
- **Arrest**: Informa si hubo o no una detención. Nos ayuda a analizar la respuesta de las autoridades ante diferentes tipos de crímenes.
- **Date**: Registra la fecha exacta del crimen. Sirve para hacer análisis temporales, identificar patrones por día, mes o incluso por horarios.
- **Location Description**: Describe el tipo de lugar donde ocurrió el delito (por ejemplo, calle, casa, escuela). Esto aporta un contexto espacial muy valioso.
- **Year**: Permite agrupar los casos por año y ver cómo han cambiado los patrones delictivos con el tiempo.

Estas columnas fueron elegidas porque ofrecen una base sólida para hacer un análisis amplio y útil, sin complicar demasiado los datos. Nos dan información suficiente para detectar tendencias, comparar situaciones y empezar a responder preguntas clave sobre el fenómeno delictivo en general.


In [9]:
from pyspark.sql.functions import col

# Revisar rangos de año y fecha
muestra_final.select("Year").describe().show()
muestra_final.agg({"Year": "min", "Year": "max"}).show()
muestra_final.agg({"Date_ts": "min", "Date_ts": "max"}).show()
# Verificaremos los valores nulos por columna clave
for columna in columnas_clave:
    n_nulos = muestra_total.filter(col(columna).isNull()).count()
    print(f"Columna '{columna}': {n_nulos} valores nulos")

# Elimina las filas con al menos un nulo en las columnas clave seleccionadas
muestra_total_sin_nulos = muestra_total.dropna(subset=columnas_clave)

print(f"\nRegistros antes de eliminar nulos: {muestra_total.count()}")
print(f"Registros después de eliminar nulos: {muestra_total_sin_nulos.count()}")

Columna 'Primary Type': 0 valores nulos


Columna 'Domestic': 0 valores nulos


Columna 'Arrest': 0 valores nulos


Columna 'Date': 0 valores nulos


Columna 'Year': 0 valores nulos


Columna 'Location Description': 86 valores nulos



Registros antes de eliminar nulos: 522964


[Stage 101:==================================================>  (133 + 7) / 140]

Registros después de eliminar nulos: 522878


In [10]:
muestra_total_sin_nulos.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [12]:
from pyspark.sql.functions import to_timestamp

# Convierte columna 'Date' a tipo timestamp (creará nueva columna 'Date_ts')
muestra_final = muestra_total_sin_nulos.withColumn(
    "Date_ts",
    to_timestamp("Date", "MM/dd/yyyy hh:mm:ss a")
)

# Opcional: elimina la columna original si solo quieres la nueva
# muestra_final = muestra_final.drop("Date").withColumnRenamed("Date_ts", "Date")

# Checa el resultado:
muestra_final.select("Date", "Date_ts").show(3, truncate=False)

+----------------------+-------------------+
|Date                  |Date_ts            |
+----------------------+-------------------+
|09/07/2015 03:00:00 AM|2015-09-07 03:00:00|
|09/08/2015 05:00:00 PM|2015-09-08 17:00:00|
|10/13/2019 05:00:00 AM|2019-10-13 05:00:00|
+----------------------+-------------------+
only showing top 3 rows



## Corrección de formatos y tipos de datos

La revisión de los tipos de datos en las columnas clave mostró que la mayoría estaban en un formato adecuado, salvo la columna **`Date`**, que se encontraba como tipo *string*. Esta columna fue convertida al tipo **fecha-hora (`timestamp`)** usando la función to_timestamp de PySpark, permitiendo así futuros análisis temporales y segmentaciones por mes, día, hora, etc.

El resto de las variables principales ya parecen estar con el tipo correcto.


In [13]:
# Revisar rangos de año y fecha
muestra_final.select("Year").describe().show()
muestra_final.agg({"Year": "min", "Year": "max"}).show()
muestra_final.agg({"Date_ts": "min", "Date_ts": "max"}).show()

+-------+------------------+
|summary|              Year|
+-------+------------------+
|  count|            522878|
|   mean| 2009.869242920911|
| stddev|6.2519741986148825|
|    min|              2001|
|    max|              2023|
+-------+------------------+



+---------+
|max(Year)|
+---------+
|     2023|
+---------+



[Stage 111:===================================================> (136 + 4) / 140]

+-------------------+
|       max(Date_ts)|
+-------------------+
|2023-05-29 22:00:00|
+-------------------+



In [14]:
muestra_final.select("Latitude", "Longitude").describe().show()
muestra_final.agg(
    {"Latitude": "min", "Latitude": "max", "Longitude": "min", "Longitude": "max"}
).show()

+-------+-------------------+-------------------+
|summary|           Latitude|          Longitude|
+-------+-------------------+-------------------+
|  count|             518172|             518172|
|   mean| 41.842532236626866| -87.67135204858764|
| stddev|0.08889303854151867|0.06112646809451275|
|    min|       36.619446395|      -91.686565684|
|    max|         42.0226409|      -87.524529378|
+-------+-------------------+-------------------+



[Stage 117:===================================================> (137 + 3) / 140]

+-------------+--------------+
|max(Latitude)|max(Longitude)|
+-------------+--------------+
|   42.0226409| -87.524529378|
+-------------+--------------+



In [15]:
# Filtramos registros con latitud y longitud dentro de Chicago
muestra_final_limpia = muestra_final.filter(
    ((col('Latitude') >= 41.6) & (col('Latitude') <= 42.1)) &
    ((col('Longitude') >= -88) & (col('Longitude') <= -87.5))
)

print('Registros antes de filtrar:', muestra_final.count())
print('Registros después de filtrar ubicaciones fuera de rango:', muestra_final_limpia.count())

Registros antes de filtrar: 522878


[Stage 123:===================================================> (136 + 4) / 140]

Registros después de filtrar ubicaciones fuera de rango: 518162


In [16]:
from pyspark.ml.feature import StringIndexer

# Indexar Primary Type y Location Description
indexer_pt = StringIndexer(inputCol="Primary Type", outputCol="PrimaryType_index")
indexer_ld = StringIndexer(inputCol="Location Description", outputCol="LocationDesc_index")

muestra_indexeada = indexer_pt.fit(muestra_final).transform(muestra_final)
muestra_indexeada = indexer_ld.fit(muestra_indexeada).transform(muestra_indexeada)

muestra_indexeada.select("Primary Type", "PrimaryType_index", "Location Description", "LocationDesc_index").show(5)

+------------+-----------------+--------------------+------------------+
|Primary Type|PrimaryType_index|Location Description|LocationDesc_index|
+------------+-----------------+--------------------+------------------+
|       THEFT|              0.0|           RESIDENCE|               1.0|
|       THEFT|              0.0|           RESIDENCE|               1.0|
|       THEFT|              0.0|            SIDEWALK|               3.0|
|       THEFT|              0.0|              STREET|               0.0|
|       THEFT|              0.0|           APARTMENT|               2.0|
+------------+-----------------+--------------------+------------------+
only showing top 5 rows



In [17]:
from pyspark.sql.functions import month, dayofweek, hour

# Creamos nuevas columnas de mes, día de la semana y hora
muestra_ext = muestra_indexeada \
    .withColumn("Month", month("Date_ts")) \
    .withColumn("DayOfWeek", dayofweek("Date_ts")) \
    .withColumn("Hour", hour("Date_ts"))

# Visualizamos un par de filas para ver las nuevas columnas
muestra_ext.select("Date_ts", "Month", "DayOfWeek", "Hour").show(5)

+-------------------+-----+---------+----+
|            Date_ts|Month|DayOfWeek|Hour|
+-------------------+-----+---------+----+
|2015-09-07 03:00:00|    9|        2|   3|
|2015-09-08 17:00:00|    9|        3|  17|
|2019-10-13 05:00:00|   10|        1|   5|
|2015-09-15 11:20:00|    9|        3|  11|
|2015-09-16 22:35:00|    9|        4|  22|
+-------------------+-----+---------+----+
only showing top 5 rows



## Transformación de variables temporales

A partir de la columna de fecha-hora (**`Date_ts`**), se generaron tres nuevas variables que permiten analizar con mayor detalle los patrones temporales de los delitos:

- **`Month`**: Número del mes en que ocurrió el crimen (valores de 1 a 12).
- **`DayOfWeek`**: Día de la semana en que ocurrió el crimen (1 = domingo, 7 = sábado, siguiendo el formato estándar de PySpark).
- **`Hour`**: Hora del día en que ocurrió el crimen (valores de 0 a 23).

Estas variables son útiles para detectar **patrones estacionales**, identificar **días con mayor incidencia delictiva**, y analizar las **franjas horarias con más actividad criminal**.


In [18]:
from pyspark.ml.feature import VectorAssembler

# Definimos las columnas de entrada para el modelo
columnas_features = [
    "PrimaryType_index",
    "LocationDesc_index",
    "Domestic",
    "Arrest",
    "Year",
    "Month",
    "DayOfWeek",
    "Hour"
]

# Construimos el ensamblador de vector
assembler = VectorAssembler(inputCols=columnas_features, outputCol="features")

# Transformamos el dataframe
muestra_features = assembler.transform(muestra_ext)

# Visualizamos ejemplo de features
muestra_features.select(columnas_features + ['features']).show(5, truncate=False)

+-----------------+------------------+--------+------+----+-----+---------+----+-------------------------------------+
|PrimaryType_index|LocationDesc_index|Domestic|Arrest|Year|Month|DayOfWeek|Hour|features                             |
+-----------------+------------------+--------+------+----+-----+---------+----+-------------------------------------+
|0.0              |1.0               |true    |false |2015|9    |2        |3   |[0.0,1.0,1.0,0.0,2015.0,9.0,2.0,3.0] |
|0.0              |1.0               |true    |false |2015|9    |3        |17  |[0.0,1.0,1.0,0.0,2015.0,9.0,3.0,17.0]|
|0.0              |3.0               |true    |false |2019|10   |1        |5   |[0.0,3.0,1.0,0.0,2019.0,10.0,1.0,5.0]|
|0.0              |0.0               |true    |true  |2015|9    |3        |11  |[0.0,0.0,1.0,1.0,2015.0,9.0,3.0,11.0]|
|0.0              |2.0               |true    |false |2015|9    |4        |22  |[0.0,2.0,1.0,0.0,2015.0,9.0,4.0,22.0]|
+-----------------+------------------+--------+-

## Preparación del vector de características ("features")

Para poder aplicar los modelos de aprendizaje automático en PySpark, fue necesario **convertir todas las variables relevantes a un único vector numérico**, ya que este es el formato requerido por los algoritmos.

Las variables utilizadas fueron:

- **`PrimaryType_index`**: Índice numérico que representa el tipo de crimen.
- **`LocationDesc_index`**: Índice numérico para el lugar donde ocurrió el crimen.
- **`Domestic`** y **`Arrest`**: Variables booleanas (valores `True` o `False`).
- **`Year`**, **`Month`**, **`DayOfWeek`**, **`Hour`**: Variables temporales representadas numéricamente.

Todas estas variables fueron **ensambladas en una sola columna llamada `features`** utilizando la función `VectorAssembler` de PySpark, lo que permite alimentar correctamente los modelos de machine learning.


## Parte 4: Preparación del conjunto de entrenamiento y prueba

En esta etapa, el objetivo fue dividir la muestra **M** ya preprocesada en dos subconjuntos: uno para **entrenar los modelos** y otro para **evaluar su rendimiento** de forma objetiva.

Para minimizar el riesgo de sesgo y mantener la representatividad de los datos, se utilizó una técnica de **muestreo aleatorio simple**. Esta técnica garantiza que cada registro tenga la misma probabilidad de ser asignado a cualquiera de los dos conjuntos, evitando así una distribución sesgada.

La división se hizo en proporción **70% para entrenamiento y 30% para prueba**. Esta elección permite contar con una base sólida para entrenar los modelos, sin dejar de lado un conjunto suficiente para realizar una evaluación confiable y replicable del desempeño del modelo.

Esta preparación es clave para garantizar que los resultados obtenidos en la etapa de modelado sean válidos y generalizables a nuevos datos.


In [19]:
entrenamiento, prueba = muestra_features.randomSplit([0.7, 0.3], seed=42)

print("Registros en conjunto de entrenamiento:", entrenamiento.count())
print("Registros en conjunto de prueba:", prueba.count())

Registros en conjunto de entrenamiento: 365684


[Stage 138:===================================================> (135 + 5) / 140]

Registros en conjunto de prueba: 157194


In [20]:
entrenamiento = entrenamiento.withColumnRenamed('Arrest', 'label')
prueba = prueba.withColumnRenamed('Arrest', 'label')

In [21]:
entrenamiento.select("features", "label").show(5)
prueba.select("features", "label").show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,0.0,1.0,0.0,...|false|
|[0.0,1.0,1.0,0.0,...|false|
|[0.0,2.0,1.0,0.0,...|false|
|[0.0,0.0,1.0,0.0,...|false|
|[0.0,0.0,1.0,0.0,...|false|
+--------------------+-----+
only showing top 5 rows

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[0.0,5.0,1.0,0.0,...|false|
|[0.0,1.0,1.0,0.0,...|false|
|[0.0,2.0,1.0,0.0,...|false|
|[0.0,1.0,1.0,0.0,...|false|
|[0.0,14.0,1.0,0.0...|false|
+--------------------+-----+
only showing top 5 rows



In [26]:
from pyspark.sql.functions import col

# Para entrenamiento:
entrenamiento = entrenamiento.withColumn("label", col("label").cast("int"))

# Para prueba:
prueba = prueba.withColumn("label", col("label").cast("int"))

In [30]:
entrenamiento.limit(5).toPandas()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,label,Domestic,...,Latitude,Longitude,Location,Date_ts,PrimaryType_index,LocationDesc_index,Month,DayOfWeek,Hour,features
0,10000299,HY189997,03/18/2015 09:15:00 PM,111XX S SANGAMON ST,0820,THEFT,$500 AND UNDER,STREET,0,True,...,41.691859,-87.646011,"(41.691858549, -87.646011379)",2015-03-18 21:15:00,0.0,0.0,3,4,21,"[0.0, 0.0, 1.0, 0.0, 2015.0, 3.0, 4.0, 21.0]"
1,10045903,HY234588,04/23/2015 02:00:00 PM,023XX S ST LOUIS AVE,0820,THEFT,$500 AND UNDER,RESIDENCE,0,True,...,41.849021,-87.712482,"(41.849021464, -87.712481981)",2015-04-23 14:00:00,0.0,1.0,4,5,14,"[0.0, 1.0, 1.0, 0.0, 2015.0, 4.0, 5.0, 14.0]"
2,10090145,HY278040,05/26/2015 06:00:00 AM,060XX S HARPER AVE,0820,THEFT,$500 AND UNDER,APARTMENT,0,True,...,41.784755,-87.588143,"(41.784754925, -87.588143196)",2015-05-26 06:00:00,0.0,2.0,5,3,6,"[0.0, 2.0, 1.0, 0.0, 2015.0, 5.0, 3.0, 6.0]"
3,10152897,HY338204,07/12/2015 09:00:00 PM,0000X N HAMLIN BLVD,0820,THEFT,$500 AND UNDER,STREET,0,True,...,41.881089,-87.720764,"(41.881088655, -87.720764494)",2015-07-12 21:00:00,0.0,0.0,7,1,21,"[0.0, 0.0, 1.0, 0.0, 2015.0, 7.0, 1.0, 21.0]"
4,10159232,HY348537,07/20/2015 04:30:00 PM,067XX S MORGAN ST,0810,THEFT,OVER $500,STREET,0,True,...,41.771670,-87.649434,"(41.771669908, -87.649434458)",2015-07-20 16:30:00,0.0,0.0,7,2,16,"[0.0, 0.0, 1.0, 0.0, 2015.0, 7.0, 2.0, 16.0]"


## Parte 5

# Aprendizaje Supervisado: Entrenamiento y Evaluación del Modelo

En la siguiente etapa, se entrenará un modelo de árbol de decisión utilizando el **DecisionTreeClassifier** de PySpark. El objetivo será predecir la probabilidad de arresto a partir de las variables seleccionadas y previamente procesadas.

In [31]:
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Crea el clasificador
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=200, seed=42)

# Entrena el modelo con el conjunto de entrenamiento
modelo_dt = dt.fit(entrenamiento)

# Genera predicciones
predicciones = modelo_dt.transform(prueba)

# Recomendado usar AUC para problemas binarios
evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = evaluator.evaluate(predicciones)

print("AUC (Area Under ROC) del modelo de Árbol de Decisión:", auc)

predicciones.select("features", "label", "rawPrediction", "probability", "prediction").show(5, truncate=False)

AUC (Area Under ROC) del modelo de Árbol de Decisión: 1.0


[Stage 171:>                                                        (0 + 1) / 1]

+--------------------------------------+-----+--------------+-----------+----------+
|features                              |label|rawPrediction |probability|prediction|
+--------------------------------------+-----+--------------+-----------+----------+
|[0.0,5.0,1.0,0.0,2015.0,5.0,4.0,14.0] |0    |[267306.0,0.0]|[1.0,0.0]  |0.0       |
|[0.0,1.0,1.0,0.0,2015.0,7.0,1.0,20.0] |0    |[267306.0,0.0]|[1.0,0.0]  |0.0       |
|[0.0,2.0,1.0,0.0,2015.0,7.0,5.0,0.0]  |0    |[267306.0,0.0]|[1.0,0.0]  |0.0       |
|[0.0,1.0,1.0,0.0,2015.0,7.0,2.0,22.0] |0    |[267306.0,0.0]|[1.0,0.0]  |0.0       |
|[0.0,14.0,1.0,0.0,2015.0,8.0,5.0,13.0]|0    |[267306.0,0.0]|[1.0,0.0]  |0.0       |
+--------------------------------------+-----+--------------+-----------+----------+
only showing top 5 rows



In [32]:
# Cuántos 0 y 1 hay en el conjunto de prueba
prueba.groupBy("label").count().show()
# En entrenamiento
entrenamiento.groupBy("label").count().show()

+-----+------+
|label| count|
+-----+------+
|    1| 42332|
|    0|114862|
+-----+------+



[Stage 175:===================================================> (135 + 5) / 140]

+-----+------+
|label| count|
+-----+------+
|    1| 98378|
|    0|267306|
+-----+------+



## Resultados del modelo

El árbol de decisión tuvo un AUC de 1.0, lo cual puede sonar muy bueno, pero en este caso no significa necesariamente que el modelo esté funcionando bien. Al revisar las etiquetas de la variable objetivo (label), se notó que solo alrededor del 27% de los registros correspondían a casos con arresto (valor "1"), mientras que la mayoría (73%) eran sin arresto (valor "0").

Esto indica que el modelo probablemente está aprendiendo a predecir siempre la clase más común, lo que hace que pierda capacidad para distinguir entre casos con y sin arresto.

Este resultado muestra lo importante que es revisar el balance entre clases cuando se trabaja con clasificación. En situaciones como esta, sería recomendable usar técnicas como el submuestreo de la clase mayoritaria, el sobremuestreo de la minoritaria o métricas como balanced accuracy o F1-score que se ajustan mejor a datos desbalanceados.


In [33]:
# Balanceando usando undersampling
from pyspark.sql.functions import col

# Filtra positivos y negativos
positivos = entrenamiento.filter(col("label") == 1)
negativos = entrenamiento.filter(col("label") == 0)

# Toma la misma cantidad de negativos que positivos
negativos_sampled = negativos.sample(fraction=positivos.count() / negativos.count(), seed=42)

# Junta ambos
entrenamiento_balanceado = positivos.union(negativos_sampled)

# Ajusta el árbol de decisión ahora con este set
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features", maxBins=200, seed=42)
modelo_dt_balanceado = dt.fit(entrenamiento_balanceado)
predicciones_balanceado = modelo_dt_balanceado.transform(prueba)
auc_balanceado = evaluator.evaluate(predicciones_balanceado)
print("AUC balanceado:", auc_balanceado)

[Stage 193:==================================================>  (134 + 6) / 140]

AUC balanceado: 1.0


In [34]:
from pyspark.sql.functions import col
confusion = predicciones.groupBy("label", "prediction").count().orderBy("label", "prediction")
confusion.show()

[Stage 204:===================================================> (136 + 4) / 140]

+-----+----------+------+
|label|prediction| count|
+-----+----------+------+
|    0|       0.0|114862|
|    1|       1.0| 42332|
+-----+----------+------+



In [38]:
predicciones.groupBy("prediction").count().show()

[Stage 213:===================================================> (137 + 3) / 140]

+----------+------+
|prediction| count|
+----------+------+
|       0.0|114862|
|       1.0| 42332|
+----------+------+



In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
roc_auc = evaluator.evaluate(predicciones)
print(f"Área bajo la curva ROC: {roc_auc}")

[Stage 216:===================================================> (137 + 3) / 140]

Área bajo la curva ROC: 1.0


## Resultados

El modelo de árbol de decisión que entrené dio resultados perfectos: logró clasificar correctamente todos los registros del conjunto de prueba y la métrica AUC fue de 1.0 (el valor más alto posible). A simple vista, esto podría sonar ideal, pero la verdad es que es muy raro que un modelo sea tan preciso en la vida real.

Cuando pasa algo así, lo más común es que hay algún detalle en los datos o en la preparación que está ayudando demasiado al modelo. Puede ser que los datos de entrenamiento y prueba no estén realmente bien mezclados, que las variables predictoras sean tan informativas que prácticamente le dan la solución al modelo, o que todavía exista cierto desbalance de clases que está influyendo en el resultado.

Aunque los números sean excelentes, es importante no confiarse y analizar con cuidado todo el flujo del preprocesamiento y la partición de los datos. Antes de usar este modelo para tomar decisiones importantes, mejor hacer una validación más profunda, probar otras métricas, nuevas particiones o incluso técnicas adicionales de balanceo y revisión de features.

El modelo funcionó "demasiado bien", pero eso es precisamente una señal para ser críticos y seguir cuestionando los resultados antes de sacar conclusiones definitivas.


In [40]:
print("Registros en conjunto de entrenamiento:", entrenamiento.count())
print("Registros en conjunto de prueba:", prueba.count())

Registros en conjunto de entrenamiento: 365684


[Stage 230:==================================================>  (134 + 6) / 140]

Registros en conjunto de prueba: 157194


### Aprendizaje No Supervisado: Agrupamiento con K-Means

Se aplicará el algoritmo K-Means con PySpark para realizar un agrupamiento no supervisado sobre la muestra preprocesada. El objetivo es encontrar posibles patrones ocultos y segmentar los datos en grupos que compartan características similares, utilizando el vector de características, generado en los pasos anteriores. Se establecerá el parámetro `k` como 4 para comenzar el análisis, aunque este valor podría ajustarse según los resultados obtenidos. Se evaluará la calidad de los clusters utilizando el coeficiente Silhouette y se explorarán los perfiles típicos de cada grupo.

In [42]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

muestra_completa = entrenamiento.union(prueba)

kmeans = KMeans(featuresCol="features", k=4, seed=42)
modelo_kmeans = kmeans.fit(muestra_completa)

clusters = modelo_kmeans.transform(entrenamiento)

evaluator = ClusteringEvaluator(featuresCol="features", metricName='silhouette', distanceMeasure='squaredEuclidean')
silhouette = evaluator.evaluate(clusters)
print("Coeficiente Silhouette para K-means:", silhouette)

clusters.groupBy("prediction").count().show()



25/05/25 12:55:14 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
                                                                                

Coeficiente Silhouette para K-means: 0.3899740206596606


[Stage 292:=================================================>   (132 + 8) / 140]

+----------+------+
|prediction| count|
+----------+------+
|         1| 89001|
|         3|102115|
|         2| 29152|
|         0|145416|
+----------+------+



### Resultados del Agrupamiento con K-Means

Después de entrenar el modelo K-Means con `k=4`, se evaluó la calidad de los clusters utilizando el coeficiente Silhouette, obteniendo un valor de **0.39**. Indica que los clusters formados presentan una separación aceptable, aunque no perfecta. Un silhouette mayor de 0.5 indicaría clusters muy bien definidos; en este caso, el resultado sugiere que los grupos existen, pero podrían solaparse un poco en las fronteras.

Esto muestra que algunos clusters son mucho más grandes que otros, lo cual es común cuando se usa K-Means con datos reales.

El uso de K-Means permitió segmentar el dataset en subgrupos que comparten ciertos rasgos similares, es útil para análisis exploratorios y para descubrir perfiles o tendencias dentro de la base de datos de crímenes de Chicago.

In [43]:
centros = modelo_kmeans.clusterCenters()
for idx, centro in enumerate(centros):
    print(f"Centro del cluster {idx}: {centro}")

Centro del cluster 0: [1.57104261e+00 4.25111530e+00 1.33614722e-01 3.34152181e-01
 2.00528398e+03 6.56191831e+00 4.06097127e+00 1.71281392e+01]
Centro del cluster 1: [1.32171828e+00 3.52171042e+00 2.16821255e-01 2.15044804e-01
 2.00911100e+03 6.56366137e+00 3.98949851e+00 4.22643452e+00]
Centro del cluster 2: [1.23280246e+00 4.40023506e+01 5.03214046e-02 2.25510889e-01
 2.01175911e+03 6.47248873e+00 4.07365922e+00 1.29194570e+01]
Centro del cluster 3: [1.39427201e+00 4.61117506e+00 1.97862282e-01 2.35978075e-01
 2.01652549e+03 6.48013703e+00 4.04387119e+00 1.59482837e+01]


In [44]:
# Ejemplo: ver los primeros 5 registros de cada cluster
for i in range(4):
    print(f"\nCluster {i}:")
    clusters.filter(clusters.prediction == i).select("features").show(5)


Cluster 0:


+--------------------+
|            features|
+--------------------+
|[0.0,0.0,1.0,0.0,...|
|[0.0,1.0,1.0,0.0,...|
|[0.0,2.0,1.0,0.0,...|
|[0.0,1.0,1.0,0.0,...|
|[0.0,1.0,1.0,0.0,...|
+--------------------+
only showing top 5 rows


Cluster 1:


+--------------------+
|            features|
+--------------------+
|[0.0,2.0,1.0,0.0,...|
|[0.0,2.0,1.0,0.0,...|
|[0.0,2.0,1.0,0.0,...|
|[0.0,1.0,1.0,0.0,...|
|[0.0,2.0,1.0,0.0,...|
+--------------------+
only showing top 5 rows


Cluster 2:


+--------------------+
|            features|
+--------------------+
|[0.0,31.0,1.0,0.0...|
|[0.0,36.0,1.0,0.0...|
|[0.0,27.0,1.0,0.0...|
|[0.0,39.0,1.0,0.0...|
|[0.0,66.0,1.0,0.0...|
+--------------------+
only showing top 5 rows


Cluster 3:
+--------------------+
|            features|
+--------------------+
|[0.0,0.0,1.0,0.0,...|
|[0.0,1.0,1.0,0.0,...|
|[0.0,0.0,1.0,0.0,...|
|[0.0,0.0,1.0,0.0,...|
|[0.0,2.0,1.0,0.0,...|
+--------------------+
only showing top 5 rows



In [45]:
clusters.groupBy("prediction").mean().show()

[Stage 300:====================================================>(138 + 2) / 140]

+----------+--------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+----------------------+-----------------------+-----------------+------------------+------------------+---------------+
|prediction|             avg(ID)|         avg(label)|         avg(Beat)|     avg(District)|         avg(Ward)|avg(Community Area)| avg(X Coordinate)| avg(Y Coordinate)|         avg(Year)|     avg(Latitude)|    avg(Longitude)|avg(PrimaryType_index)|avg(LocationDesc_index)|       avg(Month)|    avg(DayOfWeek)|         avg(Hour)|avg(prediction)|
+----------+--------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+----------------------+-----------------------+-----------------+------------------+-

## Conclusión

A lo largo de esta actividad se pudo aplicar y entender de manera práctica todo el flujo de trabajo de machine learning, desde la selección y preprocesamiento de un gran volumen de datos reales, hasta la implementación de modelos tanto supervisados como no supervisados en PySpark.

Aprendí la importancia de identificar y corregir registros nulos o inconsistentes, transformar tipos de datos y tratar variables categóricas correctamente para que los algoritmos puedan utilizarlas. También me di cuenta de lo fundamental que es revisar el balance entre las clases cuando se trabaja con problemas de clasificación.

Respecto al modelo supervisado, aunque el árbol de decisión logró resultados perfectos, esto ocurrió principalmente por el desbalance de clases y la estructura de las variables, por lo que interpreté estos números con precaución y exploré estrategias alternativas de balanceo y análisis de métricas adicionales.

En el ejercicio no supervisado con K-Means, pude observar cómo los algoritmos pueden encontrar patrones y clusters dentro de los datos sin necesidad de etiquetas, lo cual es valioso para análisis exploratorios o para entender la diversidad y estructura interna de un fenómeno complejo como el crimen en una ciudad grande.

Este trabajo me permitió valorar la utilidad y los límites de los algoritmos automáticos y la relevancia crítica de un buen preprocesamiento, más allá de los valores numéricos obtenidos, la experiencia me deja aprendizajes técnicos y prácticos para abordar proyectos similares en el futuro, así como una visión más crítica sobre los retos del machine learning con big data.